In [ ]:
# point timestep transposition

In [1]:
print("Loading Libraries...")
import os
import csv
import platform
import numpy as np
import pandas as pd
import dask.dataframe as dd
import math
import matplotlib.pyplot as plt
print("Loaded Libraries...")

Loading Libraries...
Loaded Libraries...


In [2]:
#Local
print("Loading directories..")
#path_post = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post'
path_acu = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/acu'
#path_plots = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/plots'
path_signal = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/signal'
path_rms = 'C:/Users/JMosiezny/Documents/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/rms'
print("Loaded directories...")

Loading directories..
Loaded directories...


In [ ]:
#PUT Workstation
print("Loading directories..") 
#path_data = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01'
#path_post = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post'
path_acu = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/acu'
#path_plots = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/plots'
path_signal = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/signal'
#path_rms = 'D:/01_DOKTORAT/13_PLGRID/noise-data/int-01-post/rms'
print("Loaded directories...")

In [ ]:
#wczytaj listę węzłów i zapisz jako listę do pętli zewnętrznej
#wczytaj listę plików i zapisz jako listę pętli wewnętrznej

In [ ]:
#start pętli zewnętrznej po liście węzłów
#zapisz numer węzła jako indeks wiersza z pliku

In [ ]:
#start pętli wewnętrznej (for file in filelist)
#wyciągnij z nazwy pliku nr timestep
#wczytaj pierwszy wiersz pierwszego pliku bez współrzędnych
#dodaj kolumnę timestep
#zapisz jako DF
#zapisz do dataframe pierwszego węzła (append)
#koniec pętli wewnętrznej

In [ ]:
#zapisz dataframe do pliku
#usuń dataframe z pamięci
#koniec pętli zewnętrznej

In [ ]:
print("Creating outer loop node list")
nodelist = list(range(1, 100+1))
#nodelist = list(range(1, 38138+1))
print("Done")
print("Creating inner loop file list")
filelist = sorted(os.listdir(path_acu))
print("Done")

In [ ]:
for i in nodelist:
    os.chdir(path_acu)
    node_ts = []
    for file in filelist:
        node = pd.read_csv(file, delimiter=",", usecols = ["sound-pressure", "sound-intensity"], skiprows=(range(1, i)), nrows=1, decimal='.')
        node_ts.append(node)
    node_ts = pd.concat(node_ts, axis=0, ignore_index=True)
    os.chdir(path_signal)
    node_ts.to_csv(str('int-01_acu_node_' + str(i) +'.dat'), sep=",")

In [3]:
def FFT(x):
    """A recursive implementation of the 1D Cooley-Tukey FFT"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")
    elif N <= 32:  # this cutoff should be optimized
        return DFT_slow(x)
    else:
        X_even = FFT(x[::2])
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + factor[:N / 2] * X_odd,
                               X_even + factor[N / 2:] * X_odd])

In [19]:
def NPFFT(x):
    T = 0.050150  # Duration in seconds
    #f0 = 100  # Fundamental frequency
    Fs = 50150  # Sampling frequency

    # Time domain signal
    t = np.arange(0, T*Fs)/Fs
    x = np.asarray(x, dtype=float)
    N = x.size

    # DFT
    X = np.fft.fft(x)
    X_db = 20*np.log10(2*np.abs(X)/N)
    #f = np.fft.fftfreq(N, 1/Fs)
    f = np.arange(0, N)*Fs/N
    return np.concatenate([X,
                           X_db,
                           f])

In [17]:
print("Loading batch data...")
os.chdir(path_acu)
batch_data = dd.read_csv('int-01*', delimiter=",", decimal='.',usecols=["nodenumber", "sound-pressure", "sound-intensity"]).groupby('nodenumber')
print("Batch data done...")

Loading batch data...
Batch data done...


In [18]:
batch_data

In [ ]:
fft_test = pd.DataFrame(batch_data.mean().compute())
fft_test

In [20]:
fft_test = pd.DataFrame(batch_data.apply(lambda x: NPFFT(x)).compute())
fft_test

C:\Users\JMosiezny\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  """Entry point for launching an IPython kernel.
C:\Users\JMosiezny\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log10
  del sys.path[0]


ValueError: Metadata inference failed in `groupby.apply(lambda)`.

Original error is below:
------------------------
ValueError('all the input arrays must have same number of dimensions',)

Traceback:
---------
  File "C:\Users\JMosiezny\Anaconda3\lib\site-packages\dask\dataframe\utils.py", line 137, in raise_on_meta_error
    yield
  File "C:\Users\JMosiezny\Anaconda3\lib\site-packages\dask\dataframe\groupby.py", line 1041, in apply
    meta = self._meta_nonempty.apply(func)
  File "C:\Users\JMosiezny\Anaconda3\lib\site-packages\pandas\core\groupby.py", line 716, in apply
    return self._python_apply_general(f)
  File "C:\Users\JMosiezny\Anaconda3\lib\site-packages\pandas\core\groupby.py", line 720, in _python_apply_general
    self.axis)
  File "C:\Users\JMosiezny\Anaconda3\lib\site-packages\pandas\core\groupby.py", line 1802, in apply
    res = f(group)
  File "<ipython-input-20-809e3673d442>", line 1, in <lambda>
    fft_test = pd.DataFrame(batch_data.apply(lambda x: NPFFT(x)).com

In [ ]:
#implement this http://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/ in dask

In [ ]:
T = 10  # Duration in seconds
f0 = 100  # Fundamental frequency
Fs = 1000  # Sampling frequency

# Time domain signal
t = np.arange(0, T*Fs)/Fs
x = np.sin(2*np.pi*f0*t)
N = x.size

# DFT
X = np.fft.fft(x)
X_db = 20*np.log10(2*np.abs(X)/N)
#f = np.fft.fftfreq(N, 1/Fs)
f = np.arange(0, N)*Fs/N

plt.plot(f, X_db)
plt.grid()
plt.show()

In [ ]:
x